### LVV-T1845: Verify accuracy of photometric transformation to physical scale

Verify that the DM system provides software to assess whether the accuracy of the transformation of internal LSST photometry to a physical scale (e.g. AB magnitudes) is less than **PA6 = 10 millimagnitudes**.

**Discussion**: This requires observations of a source of "truth" in the form of [CalSpec](https://www.stsci.edu/hst/instrumentation/reference-data-for-calibration-and-tools/astronomical-catalogs/calspec) (or similar) spectrophotometric standards that can be used to assess the accuracy. At the time of this testing, only a single CalSpec standard (C26202) has been observed in all LSST bands, and only with LSSTComCam. This notebook demonstrates the methods for C26202 as observed with LSSTComCam and included in Data Preview 1 (DP1); once sufficient LSSTCam observations become available, this analysis can be done for LSSTCam data.

In [1]:
# Generic python packages
import pylab as plt
import numpy as np
import pandas as pd
import glob
import math
import os
import gc
import warnings

# Set the environment variable to point to the rubin_sim data:
os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler

# rubin_sim-related packages
import rubin_sim.phot_utils as pt
import syseng_throughputs as st
from rubin_sim.data import get_data_dir

# Astropy-related packages
from astropy import units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord

# Set filter warnings to "ignore" to avoid a lot of excess log messages to the screen:
warnings.filterwarnings("ignore")

/sdf/data/rubin/user/jcarlin/repos/syseng_throughputs/syseng_throughputs/bandpassUtils.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# Which repo, collection, instrument, and skymap to use.
repo = '/repo/dp1'
collections = 'LSSTComCam/DP1'

instrument = 'LSSTComCam'
skymap_name = 'lsst_cells_v1'
day_obs_start = 20241101
day_obs_end = 20241231

verbose = 1   # 0, 1, 2, ...  Larger means more output to the screen.

# Which flux to use?  psfFlux or calibFlux?
fluxName = 'psfFlux'
fluxerrName = 'psfFluxErr'
#fluxName = 'calibFlux'
#fluxerrName = 'calibFluxErr'

# CalSpec star name
star_name = 'C26202'

# Which CalSpec spectrum FITS files to to use?
sedfile_dict = {'stiswfcnic_007' : '/sdf/data/rubin/shared/calspec/2024-11-08/c26202_stiswfcnic_007.fits', 
                'mod_008'        : '/sdf/data/rubin/shared/calspec/2024-11-08/c26202_mod_008.fits'
               }

# RA, DEC of calspec star in degrees
raDeg = 53.136845833333325
decDeg = -27.86349444444444

# List of filters to examine
flist = ['u', 'g', 'r', 'i', 'z', 'y']


#### Calculate Synthetic AB magnitudes for CalSpec star, based on official filter bandpasses

Set up appropriate hardware and system from `syseng_throughputs`

In [3]:
defaultDirs = st.setDefaultDirs()
if instrument == "LSSTComCam":
    #Change detectors from (default) LSST to ComCam (ITL CCDs)
    defaultDirs['detector'] = defaultDirs['detector'].replace('/joint_minimum',
                                                              '/itl')
hardware, system = st.buildHardwareAndSystem(defaultDirs)

#### Calculate synthetic mags for C26202

These are calculated for two different calibrated spectra of the CalSpec star: 'stiswfcnic_007' is the observed HST SED, and 'mod_008' is a model SED fit to the HST observations.

In [4]:
mags = {}

# Loop through the SEDs in our sedfile dictionary
for sed_key in sedfile_dict:
    
    print(sed_key, sedfile_dict[sed_key])
    
    # Read the SED file associated with this SED
    sedfile = sedfile_dict[sed_key]
    seddata = fits.getdata(sedfile)

    # Transform the SED data into rubin_sim format
    wavelen = seddata['WAVELENGTH'] * u.angstrom.to(u.nanometer) # This is in angstroms - need in nanometers
    flambda = seddata['FLUX'] / (u.angstrom.to(u.nanometer)) # this is in erg/sec/cm^^2/ang but we want /nm     
    sed = pt.Sed(wavelen=wavelen, flambda=flambda)
    
    # Loop over the filters, calculating the synthetic mags for each filter for this SED
    mags[sed_key] = []
    for f in flist:
        # Append the synthetic mag for this filter to this mags list for this SED
        mags[sed_key].append(sed.calc_mag(system[f]))
    # Convert list of synthetic mags for this SED into a numpy array
    mags[sed_key] = np.array(mags[sed_key])

stiswfcnic_007 /sdf/data/rubin/shared/calspec/2024-11-08/c26202_stiswfcnic_007.fits
mod_008 /sdf/data/rubin/shared/calspec/2024-11-08/c26202_mod_008.fits


In [5]:
# Convert the mags numpy arrays into a pandas dataframe
df_mags = pd.DataFrame(mags, index=flist)
df_mags

,stiswfcnic_007,mod_008
u,17.572800,17.586964
g,16.691931,16.692687
r,16.362017,16.361654
i,16.260196,16.259542
z,16.243679,16.243690
y,16.238847,16.238887


#### Query the Butler for observations of the CalSpec star

In [6]:
butler = dafButler.Butler(repo, collections=collections)

Find all `visit_image`s that overlap the sky position of the CalSpec star.

In [7]:
datasetRefs = butler.query_datasets("visit_image", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

for i, ref in enumerate(datasetRefs):    
    print(i, ref.dataId)
    if ((verbose < 2) & (i >= 10)): 
        print("...")
        break

print(f"\nFound {len(datasetRefs)} visit_images")

0 {instrument: 'LSSTComCam', detector: 5, visit: 2024120500122, band: 'r', day_obs: 20241205, physical_filter: 'r_03'}
1 {instrument: 'LSSTComCam', detector: 5, visit: 2024120900310, band: 'r', day_obs: 20241209, physical_filter: 'r_03'}
2 {instrument: 'LSSTComCam', detector: 4, visit: 2024110800288, band: 'r', day_obs: 20241108, physical_filter: 'r_03'}
3 {instrument: 'LSSTComCam', detector: 5, visit: 2024110800251, band: 'r', day_obs: 20241108, physical_filter: 'r_03'}
4 {instrument: 'LSSTComCam', detector: 3, visit: 2024111900094, band: 'y', day_obs: 20241119, physical_filter: 'y_04'}
5 {instrument: 'LSSTComCam', detector: 0, visit: 2024111700154, band: 'r', day_obs: 20241117, physical_filter: 'r_03'}
6 {instrument: 'LSSTComCam', detector: 5, visit: 2024120900311, band: 'r', day_obs: 20241209, physical_filter: 'r_03'}
7 {instrument: 'LSSTComCam', detector: 3, visit: 2024120600097, band: 'r', day_obs: 20241206, physical_filter: 'r_03'}
8 {instrument: 'LSSTComCam', detector: 3, visit:

#### Create a pandas DataFrame containing the `source` info for all these `visit_image`s.

Loop over the `datasetRefs`, grab the contents of the `source` table for each `ref`, select the entries within 3 arcsec of the Calspec star, and combine them all into one big pandas DataFrame.  

In [8]:
src_list = []

# Create SkyCoord object for the coordinates of CalSpec star
ref_coord = SkyCoord(ra=raDeg*u.degree, dec=decDeg*u.degree)

for i, ref in enumerate(datasetRefs):
    
    # Retrieve sourceTable for this visit & detector...
    dataId = {'visit': ref.dataId['visit'], 'detector': ref.dataId['detector']}
    src = butler.get('source', dataId=dataId, storageClass='DataFrame')

    # Create SkyCoord object for all points in the dataframe
    src_coords = SkyCoord(ra=src['ra'].values*u.degree, 
                          dec=src['dec'].values*u.degree)

    # Calculate separations
    separations = ref_coord.separation(src_coords)

    # Create mask for points within 3.0 arcseconds
    mask_sep = separations < 3.0*u.arcsec

    # Get filtered dataframe
    nearby_good_df = src[mask_sep]

    # Include the separations in the result
    orig_columns = nearby_good_df.columns
    nearby_good_df = src[mask_sep].copy()
    nearby_good_df['separation_calspec'] = separations[mask_sep].arcsec

    # Find (and keep) the closet match within the match radius
    best_df = nearby_good_df.sort_values('separation_calspec').drop_duplicates(subset=orig_columns,
                                                                               keep='first')
    src_list.append(best_df)

    if ((verbose >= 2) | (i < 10)): 
        print(f"{i} Visit {ref.dataId['visit']}, Detector {ref.dataId['detector']}:  Retrieved catalog of {len(src)} sources.")
    if ((verbose < 2) & (i == 10)): 
        print("...")

src_all = pd.concat(src_list, ignore_index=True)

print("")
print(f"Total combined catalog contains {len(src_all)} sources.")


0 Visit 2024120500122, Detector 5:  Retrieved catalog of 24256 sources.
1 Visit 2024120900310, Detector 5:  Retrieved catalog of 16430 sources.
2 Visit 2024110800288, Detector 4:  Retrieved catalog of 24224 sources.
3 Visit 2024110800251, Detector 5:  Retrieved catalog of 27474 sources.
4 Visit 2024111900094, Detector 3:  Retrieved catalog of 8016 sources.
5 Visit 2024111700154, Detector 0:  Retrieved catalog of 15996 sources.
6 Visit 2024120900311, Detector 5:  Retrieved catalog of 16339 sources.
7 Visit 2024120600097, Detector 3:  Retrieved catalog of 20864 sources.
8 Visit 2024112700174, Detector 3:  Retrieved catalog of 23632 sources.
9 Visit 2024112700164, Detector 1:  Retrieved catalog of 19621 sources.
...

Total combined catalog contains 738 sources.


Look at the resulting table:

In [9]:
src_all

,coord_ra,coord_dec,parentSourceId,x,y,xErr,yErr,ra,dec,raErr,...,hsmShapeRegauss_flag_no_pixels,hsmShapeRegauss_flag_not_contained,hsmShapeRegauss_flag_parent_source,sky_source,visit,detector,band,physical_filter,sourceId,separation_calspec
0,53.137027,-27.863437,600438918259148785,3543.736918,3567.315109,0.003288,0.003538,53.137027,-27.863437,1.730115e-07,...,False,False,False,False,2024120500122,5,r,r_03,600438918259149727,0.612873
1,53.137028,-27.863440,0,2026.226940,2602.800131,0.004344,0.004361,53.137028,-27.863440,2.144082e-07,...,False,False,False,False,2024120900310,5,r,r_03,600456535678125066,0.612273
2,53.137028,-27.863438,0,4006.541112,2321.674127,0.002968,0.003820,53.137028,-27.863438,1.860617e-07,...,False,False,False,False,2024110800288,4,r,r_03,600320193282966861,0.613363
3,53.137029,-27.863437,0,1353.151110,3215.383978,0.002502,0.003427,53.137029,-27.863437,1.666293e-07,...,False,False,False,False,2024110800251,5,r,r_03,600320188317435946,0.617537
4,53.137030,-27.863437,600368545755824728,1477.052112,3245.809422,0.008955,0.009168,53.137030,-27.863437,4.490681e-07,...,False,False,False,False,2024111900094,3,y,y_04,600368545755825164,0.621088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,53.137028,-27.863440,0,2605.033129,1829.205358,0.003917,0.003729,53.137028,-27.863440,1.858152e-07,...,False,False,False,False,2024120100171,3,r,r_03,600421332648723289,0.611423
734,53.137028,-27.863437,600430127132181454,1047.321282,1907.988856,0.004122,0.004251,53.137028,-27.863437,2.089816e-07,...,False,False,False,False,2024120300159,5,i,i_06,600430127132182908,0.615853
735,53.137027,-27.863436,600337786541179849,3745.984108,3011.292749,0.004262,0.002625,53.137027,-27.863436,1.339006e-07,...,False,False,False,False,2024111200296,1,z,z_03,600337786541180461,0.613636
736,53.137026,-27.863438,0,1574.050219,3289.900215,0.003917,0.004194,53.137026,-27.863438,2.055084e-07,...,False,False,False,False,2024111100086,3,r,r_03,600333360309994357,0.609293


Extract only those rows with the "bad" and "saturated" flags not set, and that are flagged as "not extended".

In [10]:
# Create a mask to cull sources with "bad" measurements.
mask = (~src_all.pixelFlags_bad) & (~src_all.pixelFlags_saturated) & \
       (~src_all.extendedness_flag) # & (src_all.detect_isPrimary)

# Apply mask, keeping only the "good" measurements of `src_all`
src_all_cleaned = src_all[mask]

# # Find (and keep) the closet match within the match radius
best_df = src_all_cleaned.sort_values('separation_calspec').drop_duplicates(subset=orig_columns, keep='first')


Add mag_obs and mag_obsErr columns:

In [11]:
# Flux in nano-Janskys to AB magnitudes:
best_df['mag_obs'] = -2.5*np.log10(best_df[fluxName]) + 31.4

# Flux error in nano-Janskys to AB magnitude error:
# Factor of 2.5/math.log(10) is explained here:  https://astronomy.stackexchange.com/questions/38371/how-can-i-calculate-the-uncertainties-in-magnitude-like-the-cds-does
best_df['mag_obsErr'] = 2.5/math.log(10)*best_df[fluxerrName]/best_df[fluxName]

Display `visit`, `detector`, `band`, fluxName, fluxerrName, `mag_obs`, `mag_obsErr`, and `separation_calspec` from best_df, sorted by `visit` and `band`:

In [12]:
# Set pandas to show all rows...
if verbose > 2:
    pd.set_option("display.max_rows", None)

In [13]:
best_df[['visit', 'detector', 'band', fluxName, fluxerrName, 'mag_obs', 'mag_obsErr', 'separation_calspec']].sort_values(['visit', 'band'])

,visit,detector,band,psfFlux,psfFluxErr,mag_obs,mag_obsErr,separation_calspec
701,2024110800245,1,i,1.163431e+06,1232.708193,16.235648,0.001150,0.620885
702,2024110800245,1,i,2.686897e+03,197.466328,22.826873,0.079793,2.974573
474,2024110800246,1,r,1.052992e+06,1024.741391,16.343937,0.001057,0.618329
433,2024110800247,5,r,1.019126e+06,1020.083237,16.379430,0.001087,0.618556
237,2024110800248,5,i,1.112040e+06,1182.766556,16.284699,0.001155,0.612944
...,...,...,...,...,...,...,...,...
86,2024121000427,2,g,7.621856e+05,1491.018663,16.694848,0.002124,0.599331
686,2024121000428,5,g,7.586157e+05,1142.177782,16.699945,0.001635,0.608013
587,2024121000430,2,g,7.508944e+05,1395.953629,16.711053,0.002018,0.608997
494,2024121000433,2,g,7.482060e+05,1758.907941,16.714947,0.002552,0.595296


In [14]:
print("""Number of rows:  %d""" % (len(best_df['visit'])))

Number of rows:  668


In [15]:
# Reset pandas to its default maximum rows to print to screen
if verbose > 2:
    pd.reset_option("display.max_rows")

#### Calculate differences between the calibrated observed magnitudes and the LSST synthetic mags for the CalSpec star

In [16]:
# Group by the 'band' column in best_df calculate the counts of 'band' for each group
count_df = best_df.groupby('band')['mag_obs'].count().reset_index()

# Rename the columns for clarity
count_df = count_df.rename(columns={'mag_obs': 'n_band'})

if verbose > 2:
    count_df

In [17]:
# Group by the 'band' column in beset_df and calculate the median of 'mag_obs' for each group
median_df = best_df.groupby('band')['mag_obs'].median().reset_index()

# Rename the columns for clarity
median_df = median_df.rename(columns={'mag_obs': 'median_mag_obs'})

if verbose > 2:
    median_df

In [18]:
# Merge the count_df and merge_df dataframes based on the filter band name
combined_df = pd.merge(count_df, median_df, left_on='band', right_on='band')

if verbose > 2:
    combined_df

In [19]:
# Reset the df_mags index to turn the keys into a column
df_mags_reset = df_mags.reset_index()

# Merge the dataframes based on the filter name
combined_df = pd.merge(combined_df, df_mags_reset, left_on='band', right_on='index')

if verbose > 2:
    combined_df

In [20]:
# Calculate the differences and add the new columns
for sed_key in sedfile_dict:
    offset_name = """offset_%s""" % (sed_key)
    combined_df[offset_name] = combined_df['median_mag_obs'] - combined_df[sed_key]

if verbose > 2:
    combined_df

In [21]:
# Output final cleaned-up results...

# Define the desired order of 'band'
order = ['u', 'g', 'r', 'i', 'z', 'y']

# Remove the 'index' column
combined_df = combined_df.drop(columns=['index'])

# Reorder the dataframe based on the 'band' column
combined_df['band'] = pd.Categorical(combined_df['band'], categories=order, ordered=True)
combined_df = combined_df.sort_values('band').reset_index(drop=True)

combined_df

,band,n_band,median_mag_obs,stiswfcnic_007,mod_008,offset_stiswfcnic_007,offset_mod_008
0,u,32,17.623262,17.572800,17.586964,0.050462,0.036298
1,g,166,16.698958,16.691931,16.692687,0.007026,0.006270
2,r,188,16.367068,16.362017,16.361654,0.005051,0.005414
3,i,128,16.260782,16.260196,16.259542,0.000586,0.001240
4,z,128,16.247457,16.243679,16.243690,0.003778,0.003767
5,y,26,16.242107,16.238847,16.238887,0.003260,0.003221


In [22]:
for band in flist:
    pickband = (combined_df['band'] == band)
    offset_hst = 1000.0*combined_df['offset_stiswfcnic_007'][pickband].values
    offset_mod = 1000.0*combined_df['offset_mod_008'][pickband].values
    
    print(f"{band}-band; empirical SED offset: {offset_hst[0]:.2f} mmag; model SED offset: {offset_mod[0]:.2f} mmag")

u-band; empirical SED offset: 50.46 mmag; model SED offset: 36.30 mmag
g-band; empirical SED offset: 7.03 mmag; model SED offset: 6.27 mmag
r-band; empirical SED offset: 5.05 mmag; model SED offset: 5.41 mmag
i-band; empirical SED offset: 0.59 mmag; model SED offset: 1.24 mmag
z-band; empirical SED offset: 3.78 mmag; model SED offset: 3.77 mmag
y-band; empirical SED offset: 3.26 mmag; model SED offset: 3.22 mmag


### Results:

We have demonstrated that tools exist to calculate the accuracy of photometric transformation by comparison between measured magnitudes and expected magnitudes of a spectrophotometric flux standard. For LSSTComCam DP1 data, the offsets between calibrated `source` tables and the predicted magnitudes catalog is less than 10 mmag for all bands but the `u` band. Because this is less than the specified threshold **PA6 = 10 mmag** for all but `u` band, we deem the result of this test a **Pass**, but will assess further with LSSTCam on-sky data when it becomes available.